# BE 172: Cardiac function - Experimental measures and computational modeling

**Objective**
The objective of this exercise is to simulate the passive inflation of a frog ventricle using the finite element method, optimize the material parameters of the model to match experimental data, and visualize the stress distribution through the heart wall.

Instructor: Jeff Omens<br>
Notebook developer: Kevin Vincent<br>

Acknowledgment: Special thanks to Henrik Fornsberg for developing [Pulse](https://github.com/finsberg/pulse), an open source finite element package for solving cardiac mechanics problems built on the [FEniCS Project](https://fenicsproject.org/)

Here is an overview of the lab exercise



1. [Plot the experimental frog heart pressure-volume relationship](#exp_data)
1. [Define cardiac geometry](#geom)
1. [Simulate passive inflation of the cardiac geometry](#fea)
1. [Optimize material parameters to match experimental data](#opt)
1. [Plot the transmural stress distribution](#stress)


***
### <center>If you are new to Jupyter notebooks <a href="Intro_to_Jupyter_Notebooks.ipynb">CLICK HERE</a> for a brief introduction</center>

### Instructions for completing this exercise

This notebook will run through successfully as provided, but the results will not be correct. There are 4 places where you need to edit the code with your own values or measurements.  As you go through the exercise you'll see **<font color=red>EDIT HERE</font>** in the text cells with a description of the data to input and the specific line numbers to edit. In the subsequent code cell, you will see another `#EDIT HERE` immediately above the lines to edit.

To complete this exercise successfully you will need to:

1. Input the experimental pressure and volume data in the correct units
1. Input the heart geometry measurements in the correct units
1. Edit the maximum pressure for the finite element model inflation
1. Iteratively edit the material properties of the finite element model to match the experimental PV curve

To save the results for your lab report you can copy the numbers out of this notebook and into an excel file.  Alternatively, if you are comfortable using Jupyter notebooks and python, you cab complete your analysis and generate figures within this notebook.

#### Troubleshooting
Below are a few suggestions if you are having trouble with this notebook.

- If you need to restart the notebook, `Kernel -> Restart & Clear Output` from the menu bar above will restart the python session, delete all the variables in memory, clear the output, and allow you to start fresh from the beginning of the notebook.
- If you want to start over completely with an unedited notebook, you will need to use the terminal. Open the Jupyter Dashboard `File -> Open...`. On the upper right hand side, select the `New -> Terminal` option. In the terminal type `git fetch origin` followed by `git reset --hard origin/master`. The repository will be reset to it's original state.
- Be sure you are not over writing a previously defined variable (see Jupyter notebook introduction)
- Check that your units are all consistent
- If a simulation or step is taking too long (>10 minutes) you may be in difficult parameter regime (probably too soft).  First, stop or interrupt the kernel (the black square on the toolbar or `Kernel -> Interrupt` from the menu). Then check that your geometry is correct and increase the stiffness of the model. You may also need to `Restart & Clear Output`

In [ ]:
# DO NOT EDIT
# This cell imports neccessary packages and sets come configurations
import os
try:
    os.system('cp plotting.py /opt/conda/lib/python3.6/site-packages/dolfin/common/plotting.py')
except Exception:
    print('Error fixing plotting bug.  3D matplotlib mesh renderings may throw error')
import matplotlib.pyplot as plt
import numpy as np
import dolfin as df
import dolfin
import mshr
import pulse
import time
from pulse.geometry_utils import generate_fibers
from pulse.geometry import Microstructure, HeartGeometry, MarkerFunctions
from func import fs, rs

os.environ['OMP_NUM_THREADS'] = '1'
dolfin.parameters['linear_algebra_backend'] = 'Eigen'
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'font.size': 18})
np.set_printoptions(precision=3, suppress=True)

***
<a id="exp_data"></a>
# Experimental Frog Heart Pressure-Volume relationship

Before running the finite element simulation, plot the experimental pressure-volume data and apply the necessary calibration factors

### Step 1: Input experimental pressure volume data

**<font color=red>EDIT HERE lines 4 & 5</font>**
<br>Input the experimental pressure and volume data in the correct units

In [ ]:
pv_lab = np.zeros((8,2))

# EDIT HERE: Input the experimental pressure and volume data in correct units
pv_lab[:,0] = np.array([0.   , 0.035, 0.07 , 0.105, 0.14 , 0.175, 0.21 , 0.245]) #Volume (mL)
pv_lab[:,1] = np.array([0.  ,  0.35,  1.4 ,  2.8 ,  4.2 ,  7.  , 11.2 , 22.4]) #Pressure (mmHg)


The file was loaded into the variable `pv_lab`. Let's print out that variable to see what is in the file. 

In [ ]:
print(pv_lab)

The data has two columns.  The first column (`pv_lab[:,0])` contains volume data.  The volume data is evenly spaced because volume was the independent variable in the experiment and is set by incrementing the syringe.  The initial volume, listed as zero, does not include the starting `Vo` volume. The second column (`pv_lab[:,1]`) contains voltage measurements read from the oscilloscope.

### Step 2: Plot the experimental data

In [ ]:
plt.plot(pv_lab[:,0],pv_lab[:,1],'k-',label='Experiment')

plt.ylim([0, 40])
plt.title('Frog heart PV curve')
plt.xlabel("Volume (mL)") # Edit to add units
plt.ylabel("Pressure (mmHg)") # Edit to add units
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.legend(loc=2,frameon=False)
plt.show()

***
<a id="geom"></a>
# Define Custom Ellipsoidal Geometry

We will model the frog ventricle as a truncated, thick-walled ellipsoid.  The epicardium and endocardium will be defined by concentric ellipsoids.  The equation for an ellipsoid is below:

\begin{equation*}
\frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} = 1 
\end{equation*}

`a` will be the dimension of the ellipsoid defining the distance from the base to the apex of the heart.  `b` and `c` describe the short-axis radius of the heart.  The nested ellipsoids will be axially symmetric (`b = c`).  The outer surface of the heart is called the epicardium (`epi`), and the inner surface of the heart is called the endocardium (`endo`).  Therefore `a_endo` is the radius from the base to the inner surface of the heart wall at the apex.

### Cardiac fiber architecture
The orientation of cardiac myocytes determines the primary direction of force development and electrical propagation.   We will incorporate the most important feature of the cardiac fiber architecture in our model by including a gradient from -60 degrees on the epicardium to +60 degrees on the endocardium.

Some figure showing data of the gradient and an image 

### Step 1: Provide input measurements for the geometry

**<font color=red>EDIT HERE lines 13 - 19</font>**
<br>Input the heart geometry measurements in the correct units

In [ ]:
# Single ventricle frog heart geomery measurements
# Note: that some variable names use 'lv' or left ventricle 
# despite the frog heart being a single v

# Define the base plane - this does not need to be edited
# 0.0 cuts the ellipsoid in half; -2.0 is a full ellipsoid; 
base_x = -0.2 

# The center of the ventricular ellipsoid
center = df.Point(0.0, 0.0, 0.0)

# EDIT HERE: Input the heart geometry measurements in the correct units (cm)
# Epicardium radii (outer surface of the heart) - a is the base to apex radius. b = c 
a_epi = 1.2
b_epi = 0.6
c_epi = 0.6

# Endocardium radii (inner surface of the heart) - a is the base to apex radius. b = c 
a_endo = 1.0
b_endo = 0.4
c_endo = 0.4

fiber_angle_epi = -60
fiber_angle_endo = 60

# Some refinement level - the default of 10 is sufficient
N = 10

mesh_name = 'frog_heart_geometry'

In [ ]:
# Markers (first index is the marker, second is the topological dimension)
markers = dict(BASE=(10, 2),
               ENDO=(30, 2),
               EPI=(40, 2))

class Endo(df.SubDomain):
    def inside(self, x, on_boundary):
        return (x[0]-center.x())**2/a_endo**2 \
            + (x[1]-center.y())**2/b_endo**2 \
            + (x[2]-center.z())**2/c_endo**2 -1.15 < df.DOLFIN_EPS \
            and on_boundary

class Base(df.SubDomain):
    def inside(self, x, on_boundary):
        return x[0] - base_x < df.DOLFIN_EPS and on_boundary

class Epi(df.SubDomain):
    def inside(self, x, on_boundary):
        return (x[0]-center.x())**2/a_epi**2 \
            + (x[1]-center.y())**2/b_epi**2 \
            + (x[2]-center.z())**2/c_epi**2 - 0.85 > df.DOLFIN_EPS \
            and on_boundary

In [ ]:
# The plane cutting the base
diam = -10.0
box = mshr.Box(df.Point(base_x, 2, 2), df.Point(diam, diam, diam))
# Generate mesh

# Ventricular epicardium
el_lv = mshr.Ellipsoid(center, a_epi, b_epi, c_epi)
# LV endocardium
el_lv_endo = mshr.Ellipsoid(center, a_endo, b_endo, c_endo)

# LV geometry (subtract the smallest ellipsoid)
lv = el_lv - el_lv_endo

# LV geometry
m = lv - box

# Create mesh
mesh = mshr.generate_mesh(m, N)

# Create facet function
ffun = df.MeshFunction("size_t", mesh, 2)
ffun.set_all(0)

endo = Endo()
endo.mark(ffun, markers['ENDO'][0])
base = Base()
base.mark(ffun, markers['BASE'][0])
epi = Epi()
epi.mark(ffun, markers['EPI'][0])

# Mark mesh
for facet in df.facets(mesh):
    mesh.domains().set_marker((facet.index(), ffun[facet]), 2)

marker_functions = MarkerFunctions(ffun=ffun)

In [ ]:
# Make fiber field
fiber_params = df.Parameters("Fibers")
fiber_params.add("fiber_space", "CG_1")
# fiber_params.add("fiber_space", "Quadrature_4")
fiber_params.add("include_sheets", False)
fiber_params.add("fiber_angle_epi", fiber_angle_epi)
fiber_params.add("fiber_angle_endo", fiber_angle_endo)

try:
    fields = generate_fibers(mesh, fiber_params)
except ImportError:
    fields = []
    fields_names = []
else:
    fields_names = ['f0', 's0', 'n0']

microstructure = Microstructure(**dict(zip(fields_names, fields)))

geometry = HeartGeometry(mesh, markers=markers,
                    marker_functions=marker_functions,
                    microstructure=microstructure)

Next, let's double check that the volume of the inner ellispoid you defined is close to the initial volume from the experimental data (`Vo`).  The following cell prints the initial cavity volume.

In [ ]:
print("Initial ventricular volume = %.5f mL"%geometry.cavity_volume())

### Step 2: Visualize mesh and fibers

In [ ]:
geometry.mesh

             The mesh above is interactive.
             You can grab it with your mouse and rotate it around to inspect it.
             Additionally, press `m` key to see the vertex locations

Next we will plot the vertices (or nodes) of the finite element mesh using matplotlib. Note the dimensions of the mesh compared to your measured dimensions

In [ ]:
plt.plot(mesh.coordinates()[:,1],mesh.coordinates()[:,0]*-1,'o')
plt.axis('equal')
plt.show()

Finally, we will plot the 3D geometry and look at the fiber angles

In [ ]:
df.plot(mesh)
ax = plt.gca()
ax.view_init(elev=-67, azim=-179)
ax.set_xlim([0, 1])
ax.set_ylim([-0.7, 0.7])
ax.set_zlim([-0.7, 0.7])
ax.set_axis_off()

plt.show()

if fields:
    df.plot(fields[0])
    ax = plt.gca()
    ax.view_init(elev=-67, azim=-179)
    ax.set_xlim([0, 1])
    ax.set_ylim([-0.7, 0.7])
    ax.set_zlim([-0.7, 0.7])
    ax.set_axis_off()

***
<a id="fea"></a>
# Passive inflation of the finite element models

### Step 1: Define Simulation Parameters

**<font color=red>EDIT HERE line 2</font>**
<br>Edit the maximum pressure for the finite element model inflation

In [ ]:
#EDIT HERE: Edit the maximum pressure for the finite element model inflation
max_lvp = 2.6 # Use kPa

# Number of steps to save pressure at  
# note: the solver in Pulse is adaptive so there should not
#       be issues with the time step being too large.  This
#.      is just for saving off the pressure
steps = 10


### Step 2: Define material properties & set boundary conditions

To model the material properties of the frog myocardium, we will use a Fung-type constitutive equation that was first proposed by [Guccione et al. in 1991](https://www.ncbi.nlm.nih.gov/pubmed/2020175)

\begin{equation*}
\psi = \frac{1}{2}C(e^Q-1)
\end{equation*}
\begin{equation*}
Q = b_{f}E_{ff}^2 + b_{fs}(2E_{fs}^2 + 2E_{fn}^2) + b_{t}(E_{ss}^2+E_{nn}^2+2E_{sn}^2)
\end{equation*}


In [ ]:
matparams = pulse.Guccione.default_parameters()
matparams["C"] = .80 # kPa
matparams["bf"] = 3.0 
matparams["bt"] = 1.5
matparams["bfs"] = 1.0 
material = pulse.Guccione(parameters=matparams,
                          f0=geometry.f0,
                          s0=geometry.s0,
                          n0=geometry.n0)

In [ ]:
# Define Dirichlet boundary. Fix the base_spring
def dirichlet_bc(W):
    V = W if W.sub(0).num_sub_spaces() == 0 else W.sub(0)
    return dolfin.DirichletBC(V, dolfin.Constant((0.0, 0.0, 0.0)),
                              geometry.ffun, geometry.markers['BASE'][0])


# Traction at the bottom of the beam
lvp = dolfin.Constant(0.0)
neumann_bc = pulse.NeumannBC(traction=lvp,
                             marker=geometry.markers['ENDO'][0])

# Collect Boundary Conditions
bcs = pulse.BoundaryConditions(dirichlet=(dirichlet_bc,),
                               neumann=(neumann_bc,))


### Step 3: Run the simulated inflation

Note: The solver will output logging info below indicating the status of the simulation.  The indicator box to the left of the cell will have a asterisk while the cell is running.  If the simulation takes a long time to run (greater than 10 minutes) there may be an issue with your geometry or parameters.  Remember `Kernel -> Restart & Clear Output` will restart the notebook if you are having problems.

In [ ]:
# Create problem
problem = pulse.MechanicsProblem(geometry, material, bcs)

#initial conditions
pressures = [0.0]
volumes = [geometry.cavity_volume()]

for p in np.linspace(0, max_lvp, steps)[1:]:
    pulse.iterate.iterate(problem, lvp, p)

    pressures.append(p)
    volumes.append(geometry.cavity_volume(u=problem.state.split()[0]))
    
# Get displacement and hydrostatic pressure
u, p = problem.state.split(deepcopy=True)

### Step 4: Plot pressure-volume relationship for the finite element simulation

In [ ]:
fig = plt.figure()
plt.plot(volumes,pressures,label='sim, C = 0.8')
plt.xlabel("Volume (mL)")
plt.ylabel("Pressure (kPa)")
plt.title('FEA heart PV curve')
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.legend()
plt.show()

MODEL OUTPUT: The simulated pressure and volume data are printed below

In [ ]:
print('pressures (kPa) = ',np.array(pressures)) 
print('volumes (mL) = ', np.array(volumes))

## Compare to experimental data

In [ ]:
p_fea_1 = np.array(pressures)*7.5 # convert to mmHg
v_fea_1 = np.array(volumes)

In [ ]:
fig = plt.figure()
plt.plot(v_fea_1,p_fea_1,label='sim, C = 0.3')
plt.plot(pv_lab[:,0],pv_lab[:,1],'k-',label='Experiment')
plt.xlabel("Volume (mL)")
plt.ylabel("Pressure (mmHg)")
plt.title('Experimental and simulated PV curves')
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.legend()
plt.show()

***
<a id="opt"></a>
# Optimize material parameters 

The objective of this portion of the exercise is to optimize the stiffness of the model to match the experimental PV data.  The below cell will edit the material parameters and run the simulated inflation all in one step.

**<font color=red>EDIT HERE line 4 </font>**
<br>Iteratively edit the material properties of the finite element model to match the experimental PV curve.  You only need to edit the `C` parameter

In [ ]:
# EDIT HERE: Iteratively edit the material properties of the finite element model 
#            to match the experimental PV curve
matparams = pulse.Guccione.default_parameters()
matparams["C"] = 0.8 # kPa
matparams["bf"] = 3.0 
matparams["bt"] = 1.5
matparams["bfs"] = 1.0 
material = pulse.Guccione(parameters=matparams,
                          f0=geometry.f0,
                          s0=geometry.s0,
                          n0=geometry.n0)

# Reset the endocardial pressure boundary condition for the new sim
lvp = dolfin.Constant(0.0)
neumann_bc = pulse.NeumannBC(traction=lvp,
                             marker=geometry.markers['ENDO'][0])

# Collect Boundary Conditions
bcs = pulse.BoundaryConditions(dirichlet=(dirichlet_bc,),
                               neumann=(neumann_bc,))

# Create problem
problem = pulse.MechanicsProblem(geometry, material, bcs)

#initial conditions
pressures = [0.0]
volumes = [geometry.cavity_volume()]

# Run the simulated inflation
for p in np.linspace(0, max_lvp, steps)[1:]:
    pulse.iterate.iterate(problem, lvp, p)

    pressures.append(p)
    volumes.append(geometry.cavity_volume(u=problem.state.split()[0]))
    
# Get displacement and hydrostatic pressure
u, p = problem.state.split(deepcopy=True)

MODEL OUTPUT: The simulated pressure and volume data are printed below

In [ ]:
print('pressures (kPa) = ',np.array(pressures))
print('volumes (mL) = ', np.array(volumes))

In [ ]:
# If you want to use this cell to plot additional 
# simulated PV curves, be sure the rename the variables

p_fea_2 = np.array(pressures)*7.5
v_fea_2 = np.array(volumes)

fig = plt.figure()
plt.plot(v_fea_1,p_fea_1,label='sim, C = 0.8  b = 3, 1.5, 1')
plt.plot(v_fea_2,p_fea_2,label='sim, C = 0.8, b = 3, 1.5, 1')
plt.plot(pv_lab[:,0],pv_lab[:,1],'k-',label='Experiment')
plt.xlabel("Volume (mL)")
plt.ylabel("Pressure (mmHg)")
plt.title('Experimental and simulated PV curves')
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.legend(loc=(1.04,0))
plt.show()

Compare your simulated PV curve to the experimental data.  To try different set of material properties, return to the [Optimize material parameters](#opt) cell, edit the value for `C` and rerun the model.  You do NOT need to run the entire notebook again, just the previous three cells (starting below [Optimize material parameters](#opt))

***
<a id="stress"></a>
# Plot the transmural stress distribution

This will use the result from the most recent simulation you've run.

In [ ]:
# Calculate strain and stress tensors
F = pulse.kinematics.DeformationGradient(u)
E = pulse.kinematics.GreenLagrangeStrain(F)
# Green-Lagrange strain normal to fiber direction
Ef = dolfin.project(
    dolfin.inner(E * geometry.f0, geometry.f0),
    dolfin.FunctionSpace(geometry.mesh, "CG", 1),
    # solver_type="gmres",
)

P = material.FirstPiolaStress(F, p)
# First piola stress normal to fiber direction
Pf = dolfin.project(
    dolfin.inner(P * geometry.f0, geometry.f0),
    dolfin.FunctionSpace(geometry.mesh, "CG", 1),
)

T = material.CauchyStress(F, p)
f = F * geometry.f0 
s = F * geometry.s0
n = F * geometry.n0


# Cauchy fiber stress
Tf = dolfin.project(
    dolfin.inner(T * f, f), dolfin.FunctionSpace(geometry.mesh, "CG", 1)
)

# Cauchy sheet stress
Sf = dolfin.project(
    dolfin.inner(T * s, s), dolfin.FunctionSpace(geometry.mesh, "CG", 1)
)

# Cauchy normal stress
Nf = dolfin.project(
    dolfin.inner(T * n, n), dolfin.FunctionSpace(geometry.mesh, "CG", 1)
)

In [ ]:
# Determine the locations to calculate the stress
x = 0.0
y = 0
z_epi = np.sqrt(c_epi**2*(1-(x**2/a_epi**2)-(y**2/b_epi**2)))
z_endo = np.sqrt(c_endo**2*(1-(x**2/a_endo**2)-(y**2/b_endo**2)))

tol = 0.03
z = np.linspace(z_endo+tol,z_epi-tol,10)
points = [(x, y, z_) for z_ in z]
fiber_stress = fs(z, matparams["C"], max_lvp)
radial_stress = rs(z, matparams["C"], max_lvp)

MODEL OUTPUT: The simulated stress data are printed below

In [ ]:
print('Z axis coordinate:',z)
print('Fiber Stress:',fiber_stress)
print('Radial Stress:',radial_stress)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4),facecolor='white')
ax1.plot(z,fiber_stress)
ax1.set_xlabel('Transmural location (Endo -> Epi)')
ax1.set_ylabel('Fiber stress (kPa)')
ax2.plot(z,radial_stress)
ax2.set_xlabel('Transmural location (Endo -> Epi)')
ax2.set_ylabel('Radial stress (kPa)')
[ax.spines['right'].set_color('none') for ax in (ax1, ax2)]
[ax.spines['top'].set_color('none') for ax in (ax1, ax2)]

plt.show()